In [3]:
# --- 0.  (one‑time) mount Drive if you haven’t already ------------------------
from google.colab import drive
drive.mount('/content/drive')         # ↳ pick “g”, “MyDrive”, etc. as usual

# --- 1.  point to the *same* folder that hosts this notebook -----------------
# For a Colab notebook the working dir is the notebook’s folder once Drive
# is mounted, so Path.cwd() does the job.  Adjust manually if you prefer.

from pathlib import Path

ROOT = Path(
    "/content/drive/MyDrive/LLMs/MLLM_CW/0723_Shuffle"
) / "MultiModalOutputs" / "gemma3_12b"
print("Operating in:", ROOT)
# --- 2.  (optional) verify owner with the Drive API ---------------------------
# Colab’s local file‑system view doesn’t expose “Owner”, so we use Drive API.
# If you *know* everything in this folder is owned by Zihao Li you can skip
# this section and jump to step 3.

from google.oauth2 import service_account
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
# ‼️  Either use a service‑account JSON or do `google.colab.auth.authenticate_user()`
import google.colab.auth as auth
auth.authenticate_user()

service = build('drive', 'v3')  # uses the notebook’s OAuth creds

def owned_by_zihao(file_path: Path) -> bool:
    """Return True iff the Drive file's primary owner is 'Zihao Li'."""
    file_id = !gdrive list {file_path}
    file_id = file_id[1].split()[0] if file_id else None      # tiny wrapper
    if not file_id:
        return False
    meta = service.files().get(fileId=file_id,
                               fields='owners').execute()
    return any(o.get('displayName') == 'Zihao Li' for o in meta['owners'])

# --- 3.  rename files ---------------------------------------------------------
OLD_ID = "EvtID278_"
NEW_ID = "EvtID193_"

for f in ROOT.glob('EvtID193_*.csv'):
    print(f)
    # If you skipped step 2 just comment out the next two lines.
    if not owned_by_zihao(f):
        continue

    new_name = f.name.replace(OLD_ID, NEW_ID, 1)
    f.rename(f.with_name(new_name))
    print(f"✔️  {f.name}  →  {new_name}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Operating in: /content/drive/MyDrive/LLMs/MLLM_CW/0723_Shuffle/MultiModalOutputs/gemma3_12b
/content/drive/MyDrive/LLMs/MLLM_CW/0723_Shuffle/MultiModalOutputs/gemma3_12b/EvtID193_ZETA_2020_South_Louisiana,_LA,_Port_of.csv


IndexError: list index out of range

In [ ]:
from pathlib import Path
from googleapiclient.discovery import build
import google.colab.auth as auth

auth.authenticate_user()
drive_svc = build("drive", "v3")          # uses your OAuth token

ROOT = Path("/content/drive/MyDrive/LLMs/MLLM_CW/0723_Shuffle"
            "/MultiModalOutputs/gemma3_12b")

OLD_ID = "EvtID193_"
NEW_ID = "EvtID278_"

def owned_by_zihao(p: Path) -> bool:
    # Query Drive for this exact file name in the parent folder
    res = drive_svc.files().list(
        q=f"name='{p.name}' and trashed=false",
        fields="files(id, owners(displayName), parents)",
    ).execute()
    files = res.get("files", [])
    if not files:
        return False
    # (Optional) confirm parent matches the Colab‑visible folder
    return any(o["displayName"] == "Zihao Li" for o in files[0]["owners"])

for f in ROOT.glob(f"{OLD_ID}*.csv"):
    if not owned_by_zihao(f):
        print(f"⏭️  Skipping {f.name} (not owned by Zihao Li)")
        continue
    new_name = f.with_name(f.name.replace(OLD_ID, NEW_ID, 1))
    f.rename(new_name)
    print(f"✔️  {f.name} → {new_name.name}")


In [5]:
from pathlib import Path
from googleapiclient.discovery import build
import google.colab.auth as auth
import re

# Authenticate and initialize Drive API
auth.authenticate_user()
drive_svc = build("drive", "v3")

# Set working directory
ROOT = Path("/content/drive/MyDrive/LLMs/MLLM_CW/0723_Shuffle"
            "/MultiModalOutputs/gemma3_12b")

# Regex to extract event ID
evt_pattern = re.compile(r"^EvtID(\d+)_")

# Check if the file is owned by Zihao Li
def owned_by_zihao(p: Path) -> bool:
    res = drive_svc.files().list(
        q=f"name='{p.name}' and trashed=false",
        fields="files(id, owners(displayName), parents)",
    ).execute()
    files = res.get("files", [])
    if not files:
        return False
    return any(o["displayName"] == "Zihao Li" for o in files[0]["owners"])

# Process files
for f in ROOT.glob("EvtID*.csv"):
    match = evt_pattern.match(f.name)
    if not match:
        continue

    old_id_num = int(match.group(1))
    new_id_num = 471- old_id_num
    new_name = f.name.replace(f"EvtID{old_id_num}_", f"EvtID{new_id_num}_", 1)
    new_path = f.with_name(new_name)

    if not owned_by_zihao(f):
        print(f"⏭️  Skipping {f.name} (not owned by Zihao Li)")
        continue

    f.rename(new_path)
    print(f"✔️  {f.name} → {new_name}")

⏭️  Skipping EvtID11_BILL_2015_Port_Arthur,_TX.csv (not owned by Zihao Li)
⏭️  Skipping EvtID10_BILL_2015_Beaumont,_TX.csv (not owned by Zihao Li)
⏭️  Skipping EvtID9_BILL_2015_Houston_Port_Authority,_TX.csv (not owned by Zihao Li)
⏭️  Skipping EvtID12_BILL_2015_Lake_Charles_Harbor_District,_LA.csv (not owned by Zihao Li)
⏭️  Skipping EvtID13_BILL_2015_Brownsville,_TX.csv (not owned by Zihao Li)
⏭️  Skipping EvtID14_BILL_2015_Cleveland-Cuyahoga_Port,_OH.csv (not owned by Zihao Li)
⏭️  Skipping EvtID6_ANA_2015_South_Jersey_Port_Corp,_NJ.csv (not owned by Zihao Li)
⏭️  Skipping EvtID4_ANA_2015_Baltimore,_MD.csv (not owned by Zihao Li)
⏭️  Skipping EvtID8_ANA_2015_Boston,_MA.csv (not owned by Zihao Li)
⏭️  Skipping EvtID3_ANA_2015_Port_of_Palm_Beach_District,_FL.csv (not owned by Zihao Li)
⏭️  Skipping EvtID5_ANA_2015_Wilmington,_DE.csv (not owned by Zihao Li)
⏭️  Skipping EvtID1_ANA_2015_Port_of_Charleston,_SC.csv (not owned by Zihao Li)
⏭️  Skipping EvtID2_ANA_2015_Wilmington,_NC.csv (n